In [1]:
using Pipe

function get_input(file)
    p1, p2 = @pipe read(file, String) |> split(_, "\n\n") |> split.(_, "\n") |> map(x -> x[2:end], _) |> map.(x -> parse(Int, x), _)
end

get_input (generic function with 1 method)

In [2]:
p1, p2 = get_input("22-t.dat")

2-element Array{Array{Int64,1},1}:
 [9, 2, 6, 3, 1]
 [5, 8, 4, 7, 10]

In [3]:
function do_turn(p1, p2)
    top1 = popfirst!(p1)
    top2 = popfirst!(p2)
    
    if top1 > top2
        vcat(p1, [top1, top2]), p2
    elseif top2 > top1
        p1, vcat(p2, [top2, top1])
    end
end

do_turn (generic function with 1 method)

In [4]:
get_score(p) = @pipe enumerate(p) |> reduce.(*, _) |> reduce(+, _)

get_score (generic function with 1 method)

In [5]:
function until_ends(p1_, p2_)
    p1, p2 = copy(p1_), copy(p2_)
    while !isempty(p1) && !isempty(p2)
        p1, p2 = do_turn(p1, p2)
    end
    @pipe [p1,p2] |> filter((!)∘isempty, _) |> first |> get_score
end

until_ends (generic function with 1 method)

In [6]:
until_ends(p1, p2)

299

In [7]:
function rc(p1_, p2_)
    p1, p2 = copy(p1_), copy(p2_)
    prev = Set()
    
    ns(p1, p2, c1, c2, w) = (w == 1 ? vcat(p1, [c1, c2]) : p1), (w == 2 ? vcat(p2, [c2, c1]) : p2)
    
    while true
        if isempty(p1) || isempty(p2)
            return (length(p1) > length(p2) ? 1 : 2), p1, p2
        elseif (p1, p2) in prev
            return 1, p1, p2
        else
            push!(prev, (p1, p2))
            c1, c2 = popfirst!(p1), popfirst!(p2)
            if length(p1) >= c1 && length(p2) >= c2
                w, _, _ = rc(p1[1:c1], p2[1:c2])
                p1, p2 = ns(p1, p2, c1, c2, w)
            else
                p1, p2 = ns(p1, p2, c1, c2, c1 > c2 ? 1 : 2)
            end
        end
    end
end

rc (generic function with 1 method)

In [8]:
@time r = rc(p1, p2)
r[r[1]+1] |> get_score
#@pipe (r1, r2) |> filter((!)∘isempty, _) |> first |> get_score

  0.110973 seconds (351.81 k allocations: 16.832 MiB, 7.43% gc time)


314